In [ ]:
# %load ../env.py
%load_ext autoreload
%autoreload 2
%pylab inline
%matplotlib inline

import matplotlib.text as text
import pandas as pd
import numpy as np
import pylab as pl
import scipy as sp
import sys
import rpy2 
import os 

#Set environment variables


# Set up the local source files
TOP = '/'.join(os.getcwd().split('/')[:-2])+'/'
#TOP = "/share/home/ishah/ipynb/pb/HTTR_WF/"

LIB = TOP+'lib'
if not LIB in sys.path: 
    sys.path.insert(0,LIB)

os.environ['PYTHONPATH']=LIB


DAT_DIR = TOP + '/data/'
FIG_DIR = TOP + '/figs/'

if not os.path.exists(DAT_DIR): os.mkdir(DAT_DIR)
if not os.path.exists(FIG_DIR): os.mkdir(FIG_DIR)


from db.mongo import *

DB = openMongo(db='genra_dev_v4')

In [ ]:
DB.tox_fp.drop()

# Goal

Load toxicity information from ToxRefDB

1. ToxRefDB existing

`mongorestore -j20 -u ishah -p ishah -d genra_dev_v4 -c tox_fp tox5_fp.bson`

2. ToxRefDB v2




In [ ]:
DB.collection_names()

In [ ]:
DB.tox_fp.find_one()

In [ ]:
DB.tox_fp.find_one().keys()

## Let's look at new toxrefdb data 

In [ ]:
DB.collection_names()

In [ ]:
DB.toxrefdb2.count()
DB.toxrefdb2.find(dict(bmds={'$exists':1})).count()

In [ ]:
X=DB.toxrefdb2.find_one(dict(preferred_name={'$regex':'Bisphenol.*','$options':'i'}, ))
#X=DB.toxrefdb2.find_one()
Y = pd.DataFrame(X['pods'])
Y.head()

In [ ]:
Y.pod_type.value_counts()

In [ ]:
Y.groupby(['endpoint_category','endpoint_type']).aggregate(dict(pod_type=len))

In [ ]:
DB.toxrefdb2.distinct('pods.endpoint_category')

In [ ]:
X=DB.toxrefdb2.find_one({'bmds':{'$exists':1},'pods.endpoint_category':u'developmental'})
#X=DB.toxrefdb2.find_one()
Y = pd.DataFrame(X['bmds'])
print X['preferred_name']
Y.head()

In [ ]:
Y2 = pd.DataFrame(X['pods']).sort_values('pod_value')
Y2.query("pod_type=='loael'").head(10)

# Aggregating the LOAEL / LEL

In [ ]:
Y2.groupby(['endpoint_category','pod_type']).aggregate(dict(pod_value=min))

In [ ]:
Y.columns

In [ ]:
Y.head().T

In [ ]:
Y[[u'AIC', u'BMD', u'BMDL', u'BMDU', u'CSF', u'bmr', u'bmr_type','model_name',
    u'endpoint_category',
    u'endpoint_target', u'endpoint_type']].sort_values('BMD')

In [ ]:
Y.groupby(['endpoint_category','endpoint_type','endpoint_target','bmr_type','model_name','bmr']).aggregate(dict(BMD=min))

# Aggregating BMD

In [ ]:
Y.query("BMD>0").groupby(['endpoint_category','bmr_type','bmr']).aggregate(dict(BMD=min))